# PreProcessamentoDeDados
Este notebook tem o objetivo realizar um pré-processamento nos dados baixados. <br>
Estes processamentos são os seguintes:
- IBGE PIB
 - Deixar apenas as cidades (removendo PIB de Regiões, estados e Brasil)
 - Adicionar a coluna de Estado, sua respectiva Sigla e a Região.
 - Remover acentos dos nomes dos locais e colocar em caixa alta
 - Ajustas alguns nomes de cidades para cruzamento de bases
 
- IBGE Shapefile
 - Remover acentos dos nomes dos locais e colocar em caixa alta
 - Ajustas alguns nomes de cidades para cruzamento de bases

In [1]:
import working_dir
working_dir.set_wd()
working_dir.get_wd()

'/Users/tales.pimentel/ds/audit2022p'

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import zipfile
import shutil
import os
from src.dao import data_urls, data_globals
from src.utils import pretties as prt
from src.utils import stats, dflib

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
FILEPATH_RAW_IBGE_PIB = data_globals.FILEPATH_RAW_IBGE_PIB
FILEPATH_RAW_IBGE_SHAPEFILE_ZIP = data_globals.FILEPATH_RAW_IBGE_SHAPEFILE_ZIP
FILEPATH_PROC_ESTADOS = data_globals.FILEPATH_PROC_ESTADOS
FILEPATH_PROC_IBGE_SHAPEFILE = data_globals.FILEPATH_PROC_IBGE_SHAPEFILE
FILEPATH_PROC_IBGE_PIB = data_globals.FILEPATH_PROC_IBGE_PIB

In [5]:
# Mapa para ajustar nomes de cidades.
# Com o objetivo que fiquem identicos aos nomes nas bases do TSE
REPLACE_NOMES_MAP = {"SAO TOME DAS LETRAS": "SAO THOME DAS LETRAS",
                     "CAMACAN": "CAMACA", "GRACHO CARDOSO": "GRACCHO CARDOSO",
                     "DONA EUZEBIA": "DONA EUSEBIA",
                     "ELDORADO DO CARAJAS": "ELDORADO DOS CARAJAS",
                     "SEM-PEIXE": "SEM PEIXE", "ARES": "AREZ",
                     "SANTA IZABEL DO PARA": "SANTA ISABEL DO PARA",
                     "AMPARO DO SAO FRANCISCO": "AMPARO DE SAO FRANCISCO",
                     "SAO LUIZ DO PARAITINGA": "SAO LUIS DO PARAITINGA",
                     "ACU": "ASSU", "ESPIGAO D'OESTE": "ESPIGAO DO OESTE",
                     "ALVORADA D'OESTE": "ALVORADA DO OESTE",
                     "PASSA-VINTE": "PASSA VINTE", "MUQUEM DE SAO FRANCISCO": "MUQUEM DO SAO FRANCISCO",
                     "ITAPAGE": "ITAPAJE", "FLORINIA": "FLORINEA", "POXOREO": "POXOREU", "IGUARACI": "IGUARACY",
                     "GRAO PARA": "GRAO-PARA", "BIRITIBA-MIRIM": "BIRITIBA MIRIM", "MOJI MIRIM": "MOGI MIRIM"}

PADROES_MAP = [("-D'", " D'")]

In [6]:
COLS_PIB = ["LOCAL",
            "PIB_2010", "PIB_PER_CAPITA_2010", 
            "PIB_2011", "PIB_PER_CAPITA_2011", 
            "PIB_2012", "PIB_PER_CAPITA_2012", 
            "PIB_2013", "PIB_PER_CAPITA_2013"]

COLS_NUM_PIB = ["PIB_2013", "PIB_PER_CAPITA_2013"]

# Estados

In [7]:
estados = pd.read_csv(FILEPATH_PROC_ESTADOS, sep=";")

cols_locais = ["ESTADO", "REGIAO", "CAPITAL"]

estados[cols_locais] = estados[cols_locais].apply(lambda col : col.str.upper())
estados[cols_locais] = estados[cols_locais].apply(lambda col: dflib.remove_acento_list(col.str.upper()))

estados.sample(5)

,SIGLA,ESTADO,REGIAO,CAPITAL
5,CE,CEARA,NORDESTE,FORTALEZA
6,DF,DISTRITO FEDERAL,CENTRO OESTE,BRASILIA
20,RS,RIO GRANDE DO SUL,SUL,PORTO ALEGRE
19,RN,RIO GRANDE DO NORTE,NORDESTE,NATAL
13,PA,PARA,NORTE,BELEM


# IBGE PIB

In [8]:
pib = pd.read_csv(FILEPATH_RAW_IBGE_PIB.replace(".xls", ".csv"), skiprows=7, skipfooter=4, header=None, sep=";", engine="python")
pib.columns = COLS_PIB
pib = pib[["LOCAL"] + COLS_NUM_PIB].rename(columns={"PIB_2013": "PIB", "PIB_PER_CAPITA_2013": "PIB_PER_CAPITA"})
   
cols_locais = ["LOCAL"]
pib[cols_locais] = pib[cols_locais].apply(lambda col : col.str.upper())
pib[cols_locais] = pib[cols_locais].apply(lambda col: dflib.remove_acento_list(col.str.upper()))

pib["LOCAL"] = pib["LOCAL"].replace(REPLACE_NOMES_MAP)

for padrao in PADROES_MAP:
    pib["LOCAL"] = pib["LOCAL"].str.replace(padrao[0], padrao[1])

pib["PIB"] = pib["PIB"].str.replace(" ", "").astype(int) * 1000
pib["PIB_PER_CAPITA"] = pib["PIB_PER_CAPITA"].str.replace(" ", "").astype(float)
pib

,LOCAL,PIB,PIB_PER_CAPITA
0,NORTE,292341723000,17213.30
1,RONDONIA,31091764000,17990.69
2,ALTA FLORESTA D'OESTE,340407000,13231.01
3,ALTO ALEGRE DOS PARECIS,182830000,13222.71
4,ALTO PARAISO,213812000,10987.82
...,...,...,...
5597,VICENTINOPOLIS,252785000,31864.95
5598,VILA BOA,105410000,20093.49
5599,VILA PROPICIO,143604000,26301.12
5600,DISTRITO FEDERAL,175362791000,62859.43


In [9]:
pib_regioes = pib[(pib["LOCAL"].str.startswith("       "))]
print("pib_regioes.shape", pib_regioes.shape)

pib_estados = pib[(pib["LOCAL"].str.startswith(" ")) & (~pib["LOCAL"].str.startswith("       "))]
print("pib_estados.shape", pib_estados.shape)

pib_estados_index = pib_estados.reset_index()[["index", "LOCAL"]].rename(columns={"index": "index_inicio"})
pib_estados_index["index_fim"] = pib_estados_index["index_inicio"].shift(-1)
pib_estados_index["index_fim"] = pib_estados_index["index_fim"] - 1
pib_estados_index["index_fim"] = pib_estados_index["index_fim"].fillna(np.inf)
pib_estados_index.head()

pib_regioes.shape (5, 3)
pib_estados.shape (27, 3)


,index_inicio,LOCAL,index_fim
0,1,RONDONIA,53.0
1,54,ACRE,76.0
2,77,AMAZONAS,139.0
3,140,RORAIMA,155.0
4,156,PARA,300.0


In [10]:
# Função para cidade para seu respectivo estado
# Utiliza o índice da tabela como referência, uma vez que as cidades estão delimitadas pelo nome do seu estado

def map_estado(i, estados_i):
    for _, row in estados_i.iterrows():     
        if (i > row["index_inicio"]) & (i <= row["index_fim"]):
            
            return row["LOCAL"]
    else:
        return "ESTADO_NÃO_IDENTIFICADO"

In [11]:
pib["ESTADO"] = pib.reset_index().apply(lambda row : map_estado(row["index"], 
                                                                pib_estados_index), axis=1).str.strip()

pib = pib.drop(index=pib_estados.index)
pib = pib.drop(index=pib_regioes.index)

In [12]:
pib = pib.merge(estados, on="ESTADO", how="left")

print("Match no merge")
prt.display(stats.freq(pib["SIGLA"].isna() == False))

pib.sample(5)

Match no merge


,Absoluto,Relativo
True,5570,1.0


,LOCAL,PIB,PIB_PER_CAPITA,ESTADO,SIGLA,REGIAO,CAPITAL
162,AUGUSTO CORREA,252665000,5932.35,PARA,PA,NORTE,BELEM
3872,TATUI,3348513000,29292.24,SAO PAULO,SP,SUDESTE,SAO PAULO
4480,MORRO GRANDE,92705000,31661.54,SANTA CATARINA,SC,SUL,FLORIANOPOLIS
4008,DIAMANTE D'OESTE,84880000,16251.21,PARANA,PR,SUL,CURITIBA
5167,RIO BRILHANTE,1447048000,43374.14,MATO GROSSO DO SUL,MS,CENTRO-OESTE,CAMPO GRANDE


In [13]:
pib.loc[pib[(pib["SIGLA"] == "BA") & (pib["LOCAL"] == "SANTA TERESINHA")].index, "LOCAL"] = "SANTA TEREZINHA"

In [14]:
print("pib.shape", pib.shape)
print(FILEPATH_PROC_IBGE_PIB)
pib.to_csv(FILEPATH_PROC_IBGE_PIB, sep=";", index=False)

pib.shape (5570, 7)
data/processed/ibge_pib.csv


#### Anotações
O Brasil possui 5568 municípios, sem contar com o Distrito Federal e Fernando de Noronha. <br>
Assim que estas 5570 representações de PIB Municipal, se encaixam perfeitamente na definição.

# IBGE Shapefile dos Municípios

In [15]:
FILEPATH_RAW_IBGE_SHAPEFILE_ZIP

'data/raw/ibge/BR_Municipios_2021.zip'

In [16]:
dir_temp = "/".join(FILEPATH_RAW_IBGE_SHAPEFILE_ZIP.split("/")[0:-1]) + "/temp/"
filename_shp = FILEPATH_RAW_IBGE_SHAPEFILE_ZIP.split("/")[-1].replace(".zip", ".shp")

with zipfile.ZipFile(FILEPATH_RAW_IBGE_SHAPEFILE_ZIP, 'r') as zip_ref:    
    zip_ref.extractall(dir_temp)
    
df_gpd = gpd.read_file(dir_temp + filename_shp)
shutil.rmtree(dir_temp)

In [17]:
df_gpd["NM_MUN"] = dflib.remove_acento_list(df_gpd["NM_MUN"].str.upper())
df_gpd["NM_MUN"] = df_gpd["NM_MUN"].str.upper()

In [18]:
df_gpd["NM_MUN"] = df_gpd["NM_MUN"].replace(REPLACE_NOMES_MAP)

for padrao in PADROES_MAP:
    df_gpd["NM_MUN"] = df_gpd["NM_MUN"].str.replace(padrao[0], padrao[1])

In [19]:
print("df_gpd.shape", df_gpd.shape)
df_gpd.sample(5)

df_gpd.shape (5572, 5)


,CD_MUN,NM_MUN,SIGLA,AREA_KM2,geometry
1811,2806107,ROSARIO DO CATETE,SE,102.683,"POLYGON ((-37.01064 -10.65623, -37.01049 -10.6..."
3561,3526001,JUNQUEIROPOLIS,SP,582.565,"POLYGON ((-51.49442 -21.22086, -51.49441 -21.2..."
1891,2905156,CAETANOS,BA,767.146,"POLYGON ((-41.00005 -14.16634, -40.95585 -14.1..."
4198,4120655,QUARTO CENTENARIO,PR,321.875,"POLYGON ((-53.07392 -24.22478, -53.07388 -24.2..."
3937,4102109,ASTORGA,PR,434.792,"POLYGON ((-51.64918 -23.08853, -51.64948 -23.0..."


In [20]:
print(FILEPATH_PROC_IBGE_SHAPEFILE)
df_gpd.to_file(FILEPATH_PROC_IBGE_SHAPEFILE) 

data/processed/BR_Municipios_2021.shp


## Validação

In [21]:
df_gpd = df_gpd.merge(pib, left_on=["NM_MUN", "SIGLA"], right_on=["LOCAL", "SIGLA"],
                      how="left")

In [22]:
shape_sem_pib = df_gpd[df_gpd["PIB"].isna()]
print(f"As {len(shape_sem_pib)} cidades a seguir possuem um shapefile mas não foi encontrado PIB.")
print(f"Elas representam {round(100 * len(shape_sem_pib) / len(df_gpd), 2)}% das cidades com Shapefile.")

prt.display(shape_sem_pib)

As 5 cidades a seguir possuem um shapefile mas não foi encontrado PIB.
Elas representam 0.09% das cidades com Shapefile.


,CD_MUN,NM_MUN,SIGLA,AREA_KM2,geometry,LOCAL,PIB,PIB_PER_CAPITA,ESTADO,REGIAO,CAPITAL
364,1708254,TABOCAO,TO,624.463,"POLYGON ((-48.39248 -8.97539, -48.39222 -8.975...",NaN,NaN,NaN,NaN,NaN,NaN
1087,2401305,CAMPO GRANDE,RN,896.954,"POLYGON ((-37.22058 -5.71750, -37.21905 -5.725...",NaN,NaN,NaN,NaN,NaN,NaN
1187,2410306,SERRA CAIADA,RN,217.539,"POLYGON ((-35.74592 -6.06218, -35.72167 -6.074...",NaN,NaN,NaN,NaN,NaN,NaN
4606,4300001,LAGOA MIRIM,RS,2872.364,"POLYGON ((-52.62347 -32.14606, -52.62286 -32.1...",NaN,NaN,NaN,NaN,NaN,NaN
4607,4300002,LAGOA DOS PATOS,RS,10196.942,"POLYGON ((-51.29028 -30.00222, -51.28939 -30.0...",NaN,NaN,NaN,NaN,NaN,NaN
